CARGAMOS PDF | EXTRAEMOS TEXTOS

In [3]:
import fitz  # PyMuPDF
##PROBAR CON LLAMA

def leer_pdf(ruta_pdf):
    # Abrir el archivo PDF
    documento = fitz.open(ruta_pdf)
    texto_completo = ""
    
    # Extraer el texto de cada página
    for pagina_num in range(documento.page_count):
        pagina = documento.load_page(pagina_num)
        texto_completo += pagina.get_text("text")
    
    # Cerrar el documento
    documento.close()
    return texto_completo

def dividir_texto_por_puntos(texto):
    # Dividir el texto por puntos
    return texto.split('.')

# Ruta al archivo PDF
ruta_pdf = "c:/Users/ivija/OneDrive/Escritorio/LLM/BIO_01.pdf"

# Leer y procesar el PDF
texto = leer_pdf(ruta_pdf)
fragmentos = dividir_texto_por_puntos(texto)

# Imprimir los fragmentos obtenidos
for i, fragmento in enumerate(fragmentos):
    print(f"Fragmento {i + 1}: {fragmento.strip()}")


Fragmento 1: Universidad nacionaL deL LitoraL 
Secretaría Académica
Dirección de Articulación, Ingreso y Permanencia 
Año 2015
Biología
Conceptos básicos
ISBN en trámite
1
Programa de Ingreso UNL / Curso de Articulación Disciplinar: Biología
El misterio más maravilloso de la vida bien pudieran ser los medios por los que se 
creó tanta diversidad a partir de tan poca materia física
Fragmento 2: La biosfera, todos los orga-
nismos combinados, representa sólo una parte en diez mil millones de la masa de 
la Tierra
Fragmento 3: Se halla distribuida de forma dispersa a través de una capa de un kilómetro 
de grosor de suelo, agua y aire que se extiende sobre quinientos millones de kilóme-
tros cuadrados de superficie
Fragmento 4: Si el planeta tuviera el tamaño de un globo terráqueo 
ordinario y su superficie se observara lateralmente a la distancia del brazo extendido, 
el ojo desnudo no podría ver traza alguna de la biosfera
Fragmento 5: Y sin embargo la vida se ha 
dividido en millones de

CONVERTIMOS TEXTOS A VECTORES | MODELO DE EMBBEDING: "all-MiniLM-L6-v2"--> Usa 384 dimensionses

In [5]:

from sentence_transformers import SentenceTransformer
modelo = SentenceTransformer('all-MiniLM-L6-V2',device='cpu') ##MODELO DE EMBBEDING
input_milvus = [] ##LISTA QUE DEBEMOS DE INSERTAR EN MILVUS
for i in range(len(fragmentos)):
    chunk = fragmentos[i]
    vector = modelo.encode(fragmentos[i])
    diccionario = {
        "embedding": vector.tolist(),  # Convertimos a lista | El diccionario cuenta con campos embedding y texto, mas adelante serán definidos
        "texto": chunk
    }
    input_milvus.append(diccionario) ##AGREGAMOS EL DICCIONARIO DE FORMA({"embedding":[0001,0212..],"texto":"Las matrices son..."}) A LISTA QUE



print(input_milvus[4])
    

No sentence-transformers model found with name meta-llama/Meta-Llama-3-8B. Creating a new one with mean pooling.


OSError: You are trying to access a gated repo.
Make sure to have access to it at https://huggingface.co/meta-llama/Meta-Llama-3-8B.
401 Client Error. (Request ID: Root=1-666b9cca-26822d737988e2e6054aef2d;c1c59f6f-c758-451c-b61f-0c6d5e2d971e)

Cannot access gated repo for url https://huggingface.co/meta-llama/Meta-Llama-3-8B/resolve/main/config.json.
Access to model meta-llama/Meta-Llama-3-8B is restricted. You must be authenticated to access it.

CONECTAMOS Y CONFIGURAMOS MILVUS LITE

In [ ]:
from milvus import default_server
default_server.start()
from pymilvus import connections
connections.connect(
         host='127.0.0.1',
         port=default_server.listen_port
    )



In [ ]:
from pymilvus import FieldSchema, CollectionSchema, DataType

##Definimos los campos
fields = [
    FieldSchema(name="id", dtype=DataType.INT64, is_primary=True, auto_id=True), ##Campo id
    FieldSchema(name="embedding", dtype=DataType.FLOAT_VECTOR, dim=384),  #Campo embedding, contendrá vectores de dimension 384
    FieldSchema(name="texto", dtype=DataType.VARCHAR, max_length=5000)    ##Campo texto, este contendrá el texto
]
schema = CollectionSchema(fields=fields,enable_dynamic_field=True)




CREAMOS COLECCION

In [ ]:
from pymilvus import Collection
coleccion = Collection(name='prueba_pdf', schema=schema)

In [ ]:
from pymilvus import utility
utility.has_collection('prueba_pdf')
##utility.drop_collection('prueba_pdf')




True

PARAMETROS DE INDEXACION 

In [ ]:
parametros_indexacion={
    "index_type" : "IVF_FLAT", 
    "metric_type" : "L2", ##L2: DISTANCIA EUCLADIANA
    "params":{"nlist":4} ##specifica el número de listas que se utilizarán en el índice <- INVESTIGAR
    }

CARGAMOS COLECCION CON LOS PARAMETROS DE INDEXACION


In [ ]:
coleccion.create_index(
    field_name="embedding", ##CAMPO AL CUAL DESEAMOS INDEXAR
    index_params=parametros_indexacion
)
coleccion.load()

INSERTAMOS EMBEDINGS

In [ ]:
coleccion.insert(input_milvus)

(insert count: 552, delete count: 0, upsert count: 0, timestamp: 450445179943124994, success count: 552, err count: 0)

In [ ]:
coleccion.flush()

EMBEDDING DE LAS QUERYS

In [ ]:
consulta = "QUE ES UN ORGANISMO VIVO?"
embedding_consulta = modelo.encode(consulta)


DEFINIMOS PARAMETROS DE BUSQUEDA

In [ ]:
respuesta = coleccion.search(
    data=[embedding_consulta], ##Valor que buscamos
    anns_field="embedding", ##Campo con el que comparamos el embedding de consulta
    param={"metric_type": "L2",
           "params": {"nprobe":2}}, ##Definimos el tipo de metrica | nprobe 2 => 2 centroids INVESTIGAR ESTOS PARAMETROS POR AHORA LO DEJO DEFAULT
    limit=5, ##Limita a 3 resultados por busqueda
    output_fields=["texto"] ##Campo que queremos que devuelva
)

MOSTRAMOS RESULTADOS

In [ ]:
for i, hits in enumerate(respuesta):
    for hit in hits:   ##Iteamos sobre la respuesta y la mostramos
        print(hit.entity.get("texto"))
        print(hit.entity.id)

  ¿Qué es un organismo vivo?
Un organismo vivo es, básicamente, material físicoquímico que exhibe un alto grado 
de complejidad, puede autorregularse, posee metabolismo y se perpetúa a sí mismo a 
través del tiempo
450445139869958987

Para muchos biólogos la vida es una fase arbitraria en la creciente complejidad de 
la materia, sin una línea divisoria precisa entre lo vivo y lo no vivo
450445139869958990
 Comencemos tratando de comprender la 
complejidad de un organismo vivo
450445139869958984
 A partir de allí se inician las teorías generales 
aplicables a las propiedades de todos los organismos
450445139869958847
 Características de los seres vivos
1
450445139869959002


PASAMOS EL OUTPUT COMO CONEXTO A UN LLM Y LE REALIZAMOS LA PRGUNTA